## Camera Calibration
Finding the Camera Matrix and Coefficients

In [1]:
## Find camera matrix and Coeficients based on series of checkerboard images
# First series of images are fed to the function to detect corners 
# if the corners are found and valid then save all the object points and corresponding image points 
# then use a calibrate camera function to obtain the camera matrix and coeficients to be used for 
# undistoring all the images
#**********************************************************************************************
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import os
import csv
#%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../CarND-Advanced-Lane-Lines/camera_cal/calibration*.jpg')
#os.chdir('..')
out_path='../CarND-Advanced-Lane-Lines/output_images/'

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_size = (gray.shape[1], gray.shape[0])
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        graySize=gray.shape[::-1]
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
        Img_info=os.path.split(fname)
        name_noExtention=os.path.splitext(Img_info[1])
        path_outImags=os.path.join(out_path,"WCorners"+Img_info[1])
        cv2.imwrite(path_outImags,img)


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, graySize,None,None)
print("mtx:", mtx)
print("dist:",dist)
"""

#write into csv file
with open('calibrationFile.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    csv_writer.writerow(mtx)
    csv_writer.writerow(dist)
"""

#write to txt file 
with open('calibrationFile.txt', 'w') as f:
    f.write(name_noExtention[0]+'\n')
    f.write('mtx:'+str(mtx)+'\n'+'dist:'+str(dist)+'\n')
    #f.write('dist:'+str(dist)+'\n')

##go through images and undistort them
for fname in images:
    img = cv2.imread(fname)
    #undistort the images then save based on found mtx and dist
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    Img_info=os.path.split(fname) 
    Undistorted_info=os.path.join(out_path+("undistorted_Camera_"+Img_info[1]))   
    cv2.imwrite(Undistorted_info,dst)    


      
           
#cv2.destroyAllWindows()


mtx: [[  1.15777818e+03   0.00000000e+00   6.67113857e+02]
 [  0.00000000e+00   1.15282217e+03   3.86124583e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
dist: [[-0.24688507 -0.02373155 -0.00109831  0.00035107 -0.00259867]]


## Functions for thresholding, lane Detection & piepline 

In [5]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import os
import pickle
from functools import reduce
import pickle as pk

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


track_leftx=[]
track_rightx=[]
left_fitx=[]
right_fitx=[]
LeftPast_x=[]
RightPast_x=[]

out_path='../CarND-Advanced-Lane-Lines/output_images/'

#********************Thresholding function*****************************

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    
    if orient =='x': 
        sobel_res=cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=3)
        
    if orient =='y': 
        sobel_res=cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=3)
        
    if (orient !='y') and (orient !='x'):
        sobel_res=cv2.Sobel(gray,cv2.CV_64F,1,1)
        
    # 3) Take the absolute value of the derivative or gradient
    abs_sober_res=np.absolute(sobel_res)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sober=255*abs_sober_res/np.max(abs_sober_res)
    scaled_sober_final=np.uint8(scaled_sober)
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    masked=np.zeros_like(scaled_sober_final)
    masked[(scaled_sober_final>=thresh_min)& (scaled_sober_final<=thresh_max)]=1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    binary_output=masked
    return binary_output

    #magnitude of change of intensity
def mag_thresh(img, sobel_kernel=7, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_resX=cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobel_resY=cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Calculate the magnitude 
    Mag_sobel=np.sqrt(np.add((sobel_resX)**2,(sobel_resY)**2))
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    Mag_sobel_scaled=np.uint8(255*Mag_sobel/np.max(Mag_sobel))
    # 5) Create a binary mask where mag thresholds are met
    masked=np.zeros_like(Mag_sobel)
    masked[(Mag_sobel_scaled>mag_thresh[0]) & (Mag_sobel_scaled<mag_thresh[1])]=1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    binary_output=masked
    return binary_output

#diretion of the change in gradient
def dir_threshold(img, sobel_kernel=9, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobel_resX=cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    sobel_resY=cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sober_resX=np.absolute(sobel_resX)
    abs_sober_resY=np.absolute(sobel_resY)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    direction=np.arctan2(abs_sober_resY,abs_sober_resX)
    # 5) Create a binary mask where direction thresholds are met
    masked=np.zeros_like(direction)
    masked[(direction>=thresh[0])& (direction<=thresh[1])]=1
    # 6) Return this mask as your binary_output image
    #binary_output = np.copy(img) # Remove this line
    binary_output = masked
    return binary_output

def HLS_Transform(img, thresh=(0, 255)):   
    
    HLS_img=cv2.cvtColor(img,cv2.COLOR_RGB2HLS)
    S_HLS_img=HLS_img[:,:,2]
    S_binary=np.zeros_like(S_HLS_img)
    S_binary[(S_HLS_img>thresh[0]) & (S_HLS_img<=thresh[1])]=1
    return S_binary

#*******************Warping(Bird-eyeView)****************************************

def warper(img, src, dst):

    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    return warped

#*******************Polynomial and laneDetection**********************************

def fit_poly(img_shape, leftx, lefty, rightx, righty,imgSize):
    # This function finds the polynomial in pixel and meters

     # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/imgSize[1] # meters per pixel in y dimension
    xm_per_pix = 3.7/imgSize[0] # meters per pixel in x dimension
    
    # polynomial in pixels
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # polynomial in meters
    left_fit_meter = np.polyfit((lefty*ym_per_pix), (leftx* xm_per_pix), 2)
    right_fit_meter = np.polyfit((righty*ym_per_pix), (rightx* xm_per_pix), 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty, right_fit, left_fit,left_fit_meter,right_fit_meter


def LaneDetection_Window(img,LeftPast,RightPast,originalSize,margin=100):
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    

    if  len(LeftPast)==0 and len(RightPast)==0:
        #if there is no availible previous detection 
        # then start estimating based on 
        histogram = np.sum(img[img.shape[0]//2:,:], axis=0)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        #TO DOOOOOOO
            # HYPERPARAMETERS
        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        # margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(img.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        """
        nonzero = img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        """
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = img.shape[0] - (window+1)*window_height
            win_y_high = img.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
                
            """
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 
            """
            
            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
            
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

    else: 
        #if there is a polynomial then use the values to find a new set of windows
        #first find a x polynomial left and right , then find a window defined by a margine around that x value
        
        X_priviousPoly_left=LeftPast[0]*(nonzeroy**2) + LeftPast[1]*nonzeroy + LeftPast[2]
        X_priviousPoly_right=RightPast[0]*(nonzeroy**2) + RightPast[1]*nonzeroy + RightPast[2]
       # X_priviousPoly_left=LeftPast_x
       # X_priviousPoly_right=RightPast_x

        left_lane_inds = ((nonzerox > (X_priviousPoly_left- margin)) & (nonzerox < X_priviousPoly_left + margin))
        right_lane_inds = ((nonzerox > (X_priviousPoly_right - margin)) & (nonzerox < X_priviousPoly_right + margin))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty, right_fit, left_fit,left_fit_meter,right_fit_meter = fit_poly(img.shape, leftx, lefty, rightx, righty,originalSize)
   # LaneDetection_Visualized=Visualization(img,left_lane_inds,right_lane_inds)
    birdView= np.dstack((img, img, img))*255
    #create a x, y points from the polynomial data, remove the 
    #dimention one from the data points 
    Line_left = np.int64(np.squeeze(np.dstack((left_fitx, ploty))))
    Line_right = np.int64(np.squeeze(np.dstack((right_fitx, ploty))))
    #draw two polylines on a black imag
    cv2.polylines(birdView, [Line_left], False, (0, 0, 255), 20)
    cv2.polylines(birdView, [Line_right], False, (255, 0, 0), 20)
    cv2.imwrite(os.path.join(out_path,'bird_eye_'+fileName), birdView)

    #note right_fit is the Coeficents of polynomial 

    # finding the Curvature average  
    avgCurvature, status_curve=measure_curvature_meter(ploty,left_fit_meter,right_fit_meter,originalSize )

    return left_fitx, right_fitx, ploty,left_fit, right_fit,avgCurvature, status_curve

###************************************Curvature in Meters***************************************************
def measure_curvature_meter(ploty, left_fit, right_fit, imgSize):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    avgCurvature=[]
    ym_per_pix = 30/imgSize[1] # meters per pixel in y dimension
    #print(left_fit[0])
    ploty=ploty* ym_per_pix 
    Curvature_stats=" "
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1+(2*left_fit[0]*y_eval+left_fit[1])**2)**(1.5))/np.absolute(2*left_fit[0])
    right_curverad = ((1+(2*right_fit[0]*y_eval+right_fit[1])**2)**(1.5))/np.absolute(2*right_fit[0])
    
    if np.abs(left_fit[0]-right_fit[0])<1.2: #if less than half a km difference then avg the curvature
        avgCurvature=(left_curverad+right_curverad)/2
    else:
        Curvature_stats="Curvature Undefined!"
    
    return avgCurvature, Curvature_stats

#****************************************Main function*******************************************
# First undistort the image based on camera matrix and coeficents obtained 
# Then find a binary thresholds (sobelx, sobely, manitude, direction, HLS(S-channel))
# Once found the combination of the binary thresholds it will warped to a bird eye view of the lane
# the warped image will pass through lane detection
# lane detection outputs the linear polynomial found to be the lanes 
# the image then warped back to normal using the same warp function switching the matrix 

def LaneDetection_pipline(img_in,mtx,dist, src, dst,track_leftx=None,track_rightx=None,LeftPast=None,RightPast=None):
    margin=100
    global left_fitx, right_fitx
    xm_per_pix = 3.7/img_in[:,:,1].shape[1] # meters per pixel in x dimension
    
    #undistort the image based on camera matrix
    img=cv2.undistort(img_in, mtx, dist, None, mtx)
    
    #thresholding process (sobel, mag, direction, SChannel of HLS)
    SobelX=abs_sobel_thresh(img, orient='x', thresh_min=10, thresh_max=220)
    SobelY=abs_sobel_thresh(img, orient='y', thresh_min=10, thresh_max=220)
    mag_binary=mag_thresh(img, sobel_kernel=9, mag_thresh=(50, 220))
    dir_binary=dir_threshold(img, sobel_kernel=15, thresh=(0.8, np.pi/2))
    HLS_binary=HLS_Transform(img, thresh=(100, 255))
    combined=np.zeros_like(dir_binary)
    #finidng a binary combination of the thresholds
    combined[((SobelX==1)&(SobelY==1))|((mag_binary==1)&(dir_binary==1))|(HLS_binary==1)]=1
    mpimg.imsave(os.path.join(out_path,"Threshold_"+fileName), combined, cmap='gray')
    img_warped=warper(combined, src, dst)
    #left_fitx, right_fitx, ploty,left_lane_inds,right_lane_inds=LaneDetection_Window(img_warped, LeftPast_x=left_fitx,RightPast_x=right_fitx, margin=100)
    left_fitx, right_fitx, ploty,left_fit, right_fit,avgCurvature, status_curve =LaneDetection_Window(img_warped, LeftPast,RightPast,[img.shape[1],img.shape[0]], margin=100)

    ##Once we get the first set of fit keep track of the rest to be used for other frames 
    track_leftx.append(left_fitx)
    track_rightx.append(right_fitx)
    if track_leftx and track_rightx:
        #left_fitx=reduce(lambda a, b: a + b, track_leftx) / len(track_leftx)
        #right_fitx=reduce(lambda a, b: a + b, track_rightx) / len(track_rightx)
        left_fitx=np.mean(track_leftx[-11:],0)
        right_fitx=np.mean(track_rightx[-11:],0)
    
    #Finding the location of the vehicle wrp to the center
    Center_polynomial=right_fitx[-1]-((right_fitx[-1]-left_fitx[-1])/2)    
    off_center=(img.shape[1]/2)-Center_polynomial
    if off_center<0:
        centre_stats="Right"
    else:
        centre_stats="Left"
    String2="Vehicle is "+str(round(abs(off_center*xm_per_pix),2))+" m " + centre_stats +" of center"
    String1="Curvature Raduis(m): "+str(round(avgCurvature))+status_curve
    #visualization
    Final_img = np.zeros_like(img_in)
    #create a x, y points from the polynomial data, remove the 
    #dimention one from the data points 
    points_left = np.int64(np.squeeze(np.dstack((left_fitx, ploty))))
    points_right = np.int64(np.squeeze(np.dstack((right_fitx, ploty))))
    #draw two polylines on a black imag
    cv2.polylines(Final_img, [points_left], False, (0, 0, 255), 30)
    cv2.polylines(Final_img, [points_right], False, (255, 0, 0), 30)
    polygon_points = np.concatenate((points_right, points_left[::-1]), 0)
    cv2.fillPoly(Final_img, [polygon_points], (0, 255, 0))
    # switch the dst and SRC to get back to input image (from bird eye view to perspective view)
    Final_img2 = warper(Final_img, dst, src)
    
    #overlay the images; summs of the channels of image(summation of two matrix)
    cv2.putText(Final_img2,String1,(15,60), cv2.FONT_HERSHEY_COMPLEX, 1,(116,90,53),2,cv2.LINE_AA)
    cv2.putText(Final_img2, String2,(15,95), cv2.FONT_HERSHEY_COMPLEX, 1,(116,90,53),2,cv2.LINE_AA)
    Final_img3= cv2.addWeighted(img_in, .5, Final_img2, .5, 0.0, dtype=0)
    cv2.imwrite(os.path.join(out_path,'FinalImg_'+fileName),Final_img3)

    return Final_img3,track_leftx,track_rightx,left_fit, right_fit 

def Visualization(img,left_lane_inds,right_lane_inds):

    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((img, img, img))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    """
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    """
    cv2.imwrite(os.path.join(out_path,'FinalImg2'+fileName),out_img)
    return out_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


## Function Calling the pipeline and passing the past parameters

In [6]:
# calling the main function and passing the data from previous frames to the next for 
# searching around the previous defined lane line, instead of using histogram data 
def get_lanes(image):

    global track_leftx, track_rightx, LeftPast_x,RightPast_x,left_fit, right_fit 
    Final_img,track_leftx,track_rightx,left_fit, right_fit =LaneDetection_pipline(image,mtx,dist, src, dst,track_leftx=track_leftx,track_rightx=track_rightx,LeftPast=left_fit,RightPast=right_fit)
    
    return Final_img

## Reading the image and passing to pipline 

In [7]:
dist=np.array([[-0.24688775, -0.02373133, -0.00109842,  0.00035108, -0.00258571]])
mtx=np.array([[1.15396093e+03, 0.00000000e+00, 6.69705357e+02],
        [0.00000000e+00, 1.14802496e+03, 3.85656234e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

src = np.float32([[600,450], [680,450], [240,720],[1050,720]])
#the bird eye view of the image 
dst = np.float32([[350,0],   [930,0],  [350,720],[930,720]])
track_leftx=[]
track_rightx=[]
left_fit=[]
right_fit=[]
LeftPast_x=[]
RightPast_x=[]


images = glob.glob('../CarND-Advanced-Lane-Lines/test_images/*.jpg')
#os.chdir('..')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    #img_info[0] is path and img_info is the name.jpg
    Img_info=os.path.split(fname)
    fileName=Img_info[1]
    # fileName_extention[0] is file name without extention JPG
    fileName_extention=os.path.splitext(fileName)
    img = mpimg.imread('../CarND-Advanced-Lane-Lines/test_images/test1.jpg')
    Out_IMG=get_lanes(img)
    

### Video Processing

In [8]:
white_output = '../CarND-Advanced-Lane-Lines/output_images/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("../CarND-Advanced-Lane-Lines/project_video.mp4") #.subclip(0,5)
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(get_lanes) #NOTE: this function expects color images!!
#%time 
white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


[MoviePy] >>>> Building video ../CarND-Advanced-Lane-Lines/output_images/project_video_output.mp4
[MoviePy] Writing video ../CarND-Advanced-Lane-Lines/output_images/project_video_output.mp4


100%|█████████▉| 1260/1261 [14:14<00:00,  1.45it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../CarND-Advanced-Lane-Lines/output_images/project_video_output.mp4 

